# Extract male bias genes on the X chromosome

In [ ]:
import session_info
import pandas as pd
from pyhere import here

In [ ]:
def get_deg():
    fn = here("differential_expression/tissue_comparison/summary_table",
              "_m/differential_expression_analysis_4features_sex.txt.gz")
    df = pd.read_csv(fn, sep='\t').loc[:, ["Tissue", "Feature", "ensemblID", "Symbol", 
                                           "seqnames", "Type", "t", "Chrom_Type"]]
    return df[(df["Type"] == "Gene")].copy()

In [ ]:
df = get_deg()

In [ ]:
xci = pd.read_csv("../../_h/xci_status_hg19.txt", sep='\t')
xci["ensemblID"] = xci["Gene ID"].str.replace("\\..*", "", regex=True)
xci.head(2)

In [ ]:
xci.groupby("Combined XCI status").size()

In [ ]:
tt = df.merge(xci[(xci["Combined XCI status"] == "escape")], on="ensemblID")
tt[(tt['t'] > 0)]

#### Escaped genes are also located on the PAR regions of the Y chromosome.

In [ ]:
xlinked = df[(df['seqnames'] == 'chrX')].copy()
xx_male = df[(df['seqnames'].isin(["chrX", "chrY"])) & (df["t"] > 0)].copy()
xlinked_male = xlinked[(xlinked["t"] > 0)].copy()
xlinked_female = xlinked[(xlinked["t"] < 0)].copy()

In [ ]:
xlinked.groupby("Tissue").size()

In [ ]:
xlinked_male.groupby("Tissue").size()

In [ ]:
xlinked_female.groupby("Tissue").size()

In [ ]:
xlinked_male

In [ ]:
xlinked_male.merge(xci[["ensemblID", "Combined XCI status"]], on="ensemblID", how="left").fillna("unknown")

In [ ]:
dx = xlinked_male.merge(xci[["ensemblID", "Combined XCI status"]], on="ensemblID", how="left").fillna("unknown")
dx = dx[(dx["Combined XCI status"] == "unknown")].copy()

In [ ]:
pd.concat([xx_male.merge(xci[["ensemblID", "Combined XCI status"]], on="ensemblID"), dx], axis=0)\
  .sort_values(["Tissue",  "Combined XCI status", "seqnames"], ascending=True)\
  .to_csv("BrainSeq_male_biased_genes_XCI_status.tsv", sep='\t', index=False)

## Session information

In [ ]:
session_info.show()